In [43]:
import glob
import nltk
import re

import pandas as pd

from string import punctuation

# Tratamento de dados coletados sobre proposições legislativas

Os dados referentes às proposições legislativas foram coletados manualmente a partir de arquivos estáticos disponíveis em https://dadosabertos.camara.leg.br/swagger/api.html#staticfile. Então, antes de iniciar seu tratamento, reuniremos todos os arquivos em um único dataframe pandas:

In [44]:
lista_proposicoes = glob.glob('../dados/proposicoes/*')

In [45]:
tipos_dados = {
    'id': object,
    'uri': object,
    'siglaTipo': object,
    'numero': object,
    'ano': int,
    'codTipo': object,
    'descricaoTipo': object,
    'ementa': object,
    'ementaDetalhada': object,
    'keywords': object,   
    'uriOrgaoNumerador': object,
    'uriPropAnterior': object,
    'uriPropPrincipal': object,
    'uriPropPosterior': object,
    'urlInteiroTeor': object, 
    'urnFinal': object, 
    'ultimoStatus_sequencia': object,
    'ultimoStatus_uriRelator': object, 
    'ultimoStatus_idOrgao': object,
    'ultimoStatus_siglaOrgao': object, 
    'ultimoStatus_uriOrgao': object,
    'ultimoStatus_regime': object, 
    'ultimoStatus_descricaoTramitacao': object,
    'ultimoStatus_idTipoTramitacao': object, 
    'ultimoStatus_descricaoSituacao': object,
    'ultimoStatus_idSituacao': object, 
    'ultimoStatus_despacho': object, 
    'ultimoStatus_url': object
}

tipo_data = ['dataApresentacao', 'ultimoStatus_dataHora']

In [46]:
lista_df = []

for proposicao in lista_proposicoes:
    df_proposicao = pd.read_csv(proposicao, sep=';', dtype=tipos_dados, parse_dates=tipo_data)
    lista_df.append(df_proposicao)

In [47]:
df_proposicao_1934_2021 = pd.concat(lista_df, axis=0, ignore_index=True)
df_proposicao_1934_2021.shape

(680358, 30)

## Seleção de dados referentes aos tipos de proposta legislativa desejados para análise

Selecionaremos apenas as propostas referentes aos seguintes tipos:
- Projeto de Decreto Legislativo [SF] (PDL)
- Projeto de Decreto Legislativo [CD] (PDC)
- Projeto de Decreto Legislativo [CN] (PDN)
- Projeto de Decreto Legislativo [SF] (PDS)
- Proposta de Emenda à Constituição (PEC)
- Projeto de Lei (PL)
- Projeto de Lei da Câmara (PLC)
- Projeto de Lei Complementar (PLP)
- Projeto de Lei de Conversão (PLV)
- Projeto de Resolução da Câmara dos Deputados (PRC)

In [48]:
tipos_proposicoes = ['PDS', 'PDC', 'PDN', 'PDS', 'PEC', 'PL', 'PLC', 'PLP', 'PLV', 'PRC']

In [49]:
df_proposicoes_tipos_desejados = df_proposicao_1934_2021[df_proposicao_1934_2021['siglaTipo'].isin(tipos_proposicoes)].copy()

In [50]:
df_proposicoes_tipos_desejados.shape

(152424, 30)

# Seleção de atributos desejados para análise

In [51]:
df_proposicoes = df_proposicoes_tipos_desejados[['id','siglaTipo','ano', 'codTipo', 'descricaoTipo',
       'ementa', 'ementaDetalhada', 'keywords']].copy()

In [52]:
df_proposicoes.shape

(152424, 8)

# Ajuste de valores faltantes

In [53]:
df_proposicoes.isnull().sum(axis = 0)

id                      0
siglaTipo               0
ano                     0
codTipo                 0
descricaoTipo           0
ementa                  5
ementaDetalhada    109844
keywords              924
dtype: int64

In [54]:
df_proposicoes[(df_proposicoes['ementa'].isnull()) & (df_proposicoes['ementaDetalhada'].isnull()) & (df_proposicoes['keywords'].isnull())].head()

,id,siglaTipo,ano,codTipo,descricaoTipo,ementa,ementaDetalhada,keywords
59695,537493,PL,1946,139,Projeto de Lei,NaN,NaN,NaN
71533,347964,PL,1935,139,Projeto de Lei,NaN,NaN,NaN
424157,2123532,PLC,2017,465,Projeto de Lei da Câmara dos Deputados (SF),NaN,NaN,NaN
495917,2308959,PLV,2021,390,Projeto de Lei de Conversão,NaN,NaN,NaN
571961,605771,PL,1954,139,Projeto de Lei,NaN,NaN,NaN


In [55]:
df_proposicoes[(df_proposicoes['ementa'].isnull())].head()

,id,siglaTipo,ano,codTipo,descricaoTipo,ementa,ementaDetalhada,keywords
59695,537493,PL,1946,139,Projeto de Lei,NaN,NaN,NaN
71533,347964,PL,1935,139,Projeto de Lei,NaN,NaN,NaN
424157,2123532,PLC,2017,465,Projeto de Lei da Câmara dos Deputados (SF),NaN,NaN,NaN
495917,2308959,PLV,2021,390,Projeto de Lei de Conversão,NaN,NaN,NaN
571961,605771,PL,1954,139,Projeto de Lei,NaN,NaN,NaN


In [56]:
df_proposicoes.dropna(axis=0, subset=['ementa'], inplace=True)

In [57]:
df_proposicoes.shape

(152419, 8)

# Extração de palavras chaves das ementas, quando necessário

Identificação de propostas legislativas com campo "keywords" vazio

In [58]:
df_proposicoes_sem_keywords = df_proposicoes[df_proposicoes['keywords'].isna()].copy()

Processamento da coluna "ementa" para extração de palavras chaves e preenchimento da coluna "keywords"

In [59]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/cecivieira/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/cecivieira/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Remoção de pontuação, preposições e artigos (stopwords)

In [60]:
meses = ['janeiro', 'fevereiro', 'março', 'abril', 'maio', 'junho', 'julho','agosto', 'setembro', 'outubro', 'novembro', 'dezembro']

In [61]:
def define_stopwords_punctuation():
    stopwords = nltk.corpus.stopwords.words('portuguese') + meses
    pontuacao = list(punctuation)
    stopwords.extend(pontuacao)
    return stopwords

In [62]:
def remove_stopwords_punctuation_da_sentenca(texto):
    padrao_digitos = r'[0-9]'
    texto = re.sub(padrao_digitos, '', texto)
    palavras = nltk.tokenize.word_tokenize(texto.lower())
    stopwords = define_stopwords_punctuation()
    keywords = [palavra for palavra in palavras if palavra not in stopwords]
    return keywords

In [63]:
df_proposicoes_sem_keywords['keywords'] = df_proposicoes_sem_keywords['ementa'].apply(remove_stopwords_punctuation_da_sentenca)

Identifica caracteres e abreviações semanticamente irrelevantes ainda presentes na coluna "keywords"

In [64]:
lista_keywords = []
lista_keywords_temp = df_proposicoes_sem_keywords['keywords'].tolist()
lista_keywords_processo = [lista_keywords.extend(item) for item in lista_keywords_temp]

In [65]:
palavras_para_descarte = []
palavras_para_descarte = [item for item in set(lista_keywords) if len(item) <= 3]

Retira os substantivos da lista de caracteres e abreviações semanticamente irrelevantes

In [66]:
substantivos = ['rio', 'sul', 'cão', 'mãe', 'oab', 'boa', 'pré', 'voz', 'rui', 'uva', 'gás', 'glp', 'apa']

Remove da coluna "keywords" lista de caracteres e abreviações semanticamente irrelevantes

In [67]:
palavras_para_descarte_refinada = [palavra for palavra in palavras_para_descarte if palavra not in substantivos]

In [68]:
def remove_palavras_para_descarte_da_sentenca(texto):
    keywords = []
    for palavra in texto:
        if palavra not in palavras_para_descarte_refinada:
            keywords.append(palavra)
    return keywords

In [69]:
df_proposicoes_sem_keywords['keywords'] = df_proposicoes_sem_keywords['keywords'].apply(remove_palavras_para_descarte_da_sentenca)

Identifica, na coluna "keywords", palavras sem relevancia semântica, por exemplo: "altera", "dispõe" e "sobre".

In [70]:
def gera_n_grams(texto, ngram=2):
    temporario = zip(*[texto[indice:] for indice in range(0,ngram)])
    resultado = [' '.join(ngram) for ngram in temporario]
    return resultado

In [71]:
df_proposicoes_sem_keywords['bigrams'] = df_proposicoes_sem_keywords['keywords'].apply(gera_n_grams)

In [72]:
lista_ngrams = []
lista_ngrams_temp = df_proposicoes_sem_keywords['bigrams'].tolist()
lista_ngrams_processo = [lista_ngrams.extend(item) for item in lista_ngrams_temp]

In [73]:
bigrams_comuns = nltk.FreqDist(lista_ngrams).most_common(50)

In [74]:
lista_bigramas_comuns = [bigrama for bigrama, frequencia in bigrams_comuns]

In [75]:
lista_bigramas_comuns_limpa = ['dispõe sobre', 'outras providências', 'nova redação', 'poder executivo', 'distrito federal',
 'autoriza poder', 'federal outras','redação constituição', 'dispõe sôbre', 'código penal', 'artigo constituição',
 'disposições constitucionais', 'altera dispõe', 'decreto-lei código', 'constitucionais transitórias', 'altera redação',
 'abre ministério', 'executivo abrir', 'redação artigo', 'sobre criação', 'acrescenta parágrafo', 'parágrafo único',
 'concede isenção', 'altera dispositivos', 'altera complementar', 'dispondo sobre', 'código processo', 'outras providências.',
 'providências. historico', 'ministério fazenda', 'altera leis', 'programa nacional', 'quadro permanente', 'outras providencias',
 'inciso constituição', 'abrir ministério', 'estabelece normas', 'ministério justiça', 'tempo serviço', 'instituto nacional',
 'institui sistema', 'operações crédito', 'altera institui', 'dispõe sôbre']

In [76]:
palavras_para_descarte_origem_biogramas = []
palavras_para_descarte_origem_biogramas_processo = [palavras_para_descarte_origem_biogramas.extend(biograma.split(' ')) for biograma in lista_bigramas_comuns_limpa]
palavras_para_descarte_origem_biogramas_unicas = set(palavras_para_descarte_origem_biogramas)

In [77]:
def remove_palavras_origem_biogramas_da_sentenca(texto):
    keywords = []
    for palavra in texto:
        if palavra not in palavras_para_descarte_origem_biogramas_unicas:
            keywords.append(palavra)
    return keywords

In [78]:
df_proposicoes_sem_keywords['keywords'] = df_proposicoes_sem_keywords['keywords'].apply(remove_palavras_origem_biogramas_da_sentenca)

Elimina coluna "biograms"

In [79]:
df_proposicoes_sem_keywords = df_proposicoes_sem_keywords.drop(columns=['bigrams']).copy()

Reuni proposição com keywords retiradas da ementa com restante das proposições legislativas

In [80]:
df_proposicoes_com_keywords = df_proposicoes[df_proposicoes['keywords'].notna()].copy()

In [81]:
df_proposicoes_v_final = pd.concat([df_proposicoes_com_keywords, df_proposicoes_sem_keywords])

In [82]:
df_proposicoes_v_final.shape

(152419, 8)